In [52]:
import streamlit as st
import pandas as pd
import numpy as np
import re
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aadit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
#nlp model
df=pd.read_csv(r'spam.csv',encoding='ISO-8859-1')
df = df.iloc[:,0:2]
df = df.rename(columns = {'v1':'labels', 'v2':"Text"})
df.drop_duplicates(inplace = True)
df['labels'] = df['labels'].map({'ham':0, 'spam':1})


def clean_data(message):
    message_without_punc=[character for character in message if character not in string.punctuation]
    message_without_punc=''.join(message_without_punc)
    seperator= ' '
    return seperator.join([word for word in message_without_punc.split() if word.lower() not in stopwords.words('english')])

In [55]:
df['Text']=df['Text'].apply(clean_data)
x=df['Text']
y=df['labels']
cv=CountVectorizer()
x=cv.fit_transform(x)

In [61]:
x_train,x_test,y_train,y_test=train_test_split(x, y,test_size=0.2,random_state=0)
model=MultinomialNB().fit(x_train,y_train)
predictions=model.predict(x_test)
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.965183752417795
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       885
           1       0.85      0.92      0.88       149

    accuracy                           0.97      1034
   macro avg       0.92      0.95      0.93      1034
weighted avg       0.97      0.97      0.97      1034

[[861  24]
 [ 12 137]]


In [58]:
def predict(text):
    labels= ['Not Spam','Spam']
    x=cv.transform(text).toarray()
    p=model.predict(x)
    s=[str(i) for i in p]
    v=int(''.join(s))
    return str('This message is looking to be: '+labels[v])
    

In [60]:
st.title('Spam Classifier')
st.image('image.jpg')
user_input=st.text_input('Write your message')
submit=st.button('Predict')
if submit:
    answer=predict([user_input])
    st.text(answer)